In [31]:
from handlers.UniswapV2 import UniswapV2
from utils.ERC20 import ERC20Handler
from dotenv import load_dotenv
from IPython.display import HTML
from web3 import Web3
import numpy as np
import datetime
import pandas as pd
import os

# USDC/ETH: 0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc
# WBTC/WETH: 0xBb2b8038a1640196FbE3e38816F3e67Cba72D940

# Adjust pandas print settings
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.0f}'.format

# Connect to geth
load_dotenv()
w3 = Web3(Web3.HTTPProvider(os.getenv('WEB3_PROVIDER_HTTP')))

# Instantiate objs
uni = UniswapV2()
erc20 = ERC20Handler()

# Get token addresses in Uni v2 pool
tokens = uni.get_pool_tokens("0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc")

# Get decimal places of tokens in pool
token0decimals = erc20.get_decimals(tokens['token0'])
token1decimals = erc20.get_decimals(tokens['token1'])

# Get token symbols of tokens in pool
token0symbol = erc20.get_symbol(tokens['token0'])
token1symbol = erc20.get_symbol(tokens['token1'])

# Get reserves of token across blocks
current_block = w3.eth.block_number
data = []
for block in range(current_block - 100, current_block):
    # get reserves of uniswap pool
    reserves = uni.get_reserves("0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc", block)

    # add token symbols to dictionary
    reserves['token0'] = token0symbol
    reserves['token1'] = token1symbol

    # Calculate exchange rate of the pool
    pool_price = (reserves['reserve1'] * 10 ** token1decimals) / (reserves['reserve0'] * 10 ** token0decimals)

    # Add pool price to reserves dict
    reserves['price (USD)'] = pool_price

    reserve0 = reserves['reserve0']
    reserve1 = reserves['reserve1']
    reserves['reserve0'] = reserve1 / (10 ** token0decimals)
    reserves['reserve1'] = reserve0 / (10 ** token1decimals)

    # Convert timestamp to datetime
    reserves['timestamp'] = datetime.datetime.fromtimestamp(reserves['timestamp'])

    data.append(reserves)

# Add data to data frame
df = pd.DataFrame(data)
df['reserve0_delta'] = df['reserve0'].diff()
df['reserve1_delta'] = df['reserve1'].diff()
df['price_delta'] = df['price (USD)'].diff()

# Add volume and liquidity
if 'USDC' in token0symbol:
    df['volume (USD)'] = abs(df['reserve0_delta'])
    df['liquidity (USD)'] = df['reserve0'] + (df['reserve1'] * df['price (USD)'])
else:
    df['volume (USD)'] = abs(df['reserve1_delta'])
    df['liquidity (USD)'] = df['reserve1'] + (df['reserve0'] * df['price (USD)'])

HTML(df[1:].round(0).to_html(index=False))

block,timestamp,reserve0,reserve1,token0,token1,price (USD),reserve0_delta,reserve1_delta,price_delta,volume (USD),liquidity (USD)
14299215,2022-02-28 23:29:48,"104,529,338","35,753",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,058,677"
14299216,2022-02-28 23:29:48,"104,529,338","35,753",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,058,677"
14299217,2022-02-28 23:29:48,"104,529,338","35,753",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,058,677"
14299218,2022-02-28 23:30:52,"104,528,423","35,754",USDC                            ,WETH                            ,"2,924",-915,0,-0,915,"209,056,847"
14299219,2022-02-28 23:30:52,"104,528,423","35,754",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,056,847"
14299220,2022-02-28 23:31:07,"104,527,277","35,754",USDC                            ,WETH                            ,"2,924","-1,147",0,-0,"1,147","209,054,553"
14299221,2022-02-28 23:31:07,"104,527,277","35,754",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,054,553"
14299222,2022-02-28 23:31:07,"104,527,277","35,754",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,054,553"
14299223,2022-02-28 23:31:07,"104,527,277","35,754",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,054,553"
14299224,2022-02-28 23:31:07,"104,527,277","35,754",USDC                            ,WETH                            ,"2,924",0,0,0,0,"209,054,553"
